 # Libraries Importing 📚⬇️
 ---

In [29]:
## BASICS
import os
import numpy as np              # linear algebra
import pandas as pd             # data processing, CSV file I/O ﴾e.g. pd.read_csv﴿ 
import matplotlib.pyplot as plt
import zipfile
import seaborn as sns
import re

# METHOD 2.1
---
<center><a href="https://ibb.co/RzL3YmL"><img src="https://i.ibb.co/ZWyBGPy/Screen-Shot-2021-02-04-at-00-35-18.png" alt="Screen-Shot-2021-02-04-at-00-35-18" border="0"></a></center>


## BASICS EQUATIONS

* $\frac{1}{f} = \frac{1}{s} + \frac{1}{s'} \Rightarrow f = \frac{ss'}{s + s'} \rightarrow s' = \frac{sf}{s - f}$

* $m = - \frac{s'}{s }$

## DATA

* Distances $s_1 = 8, s_6' = 0.015 $
* We will use $n = 6$ **thin double convex** lens. Therefore, the focal point (the **image**) will always be on the right!

---

## The Equilvalent Focal Length of Multi Lens System (n = 6)

$$f = \frac{f_1  f_2  f_3  f_4  f_5  f_6}{f_1  f_2  f_3  f_4  f_5 + f_1  f_2  f_3  f_4  f_6  + f_1  f_2  f_3  f_5  f_6 + f_1  f_2  f_4 f_5 f_6 + f_1 f_3 f_4 f_5 f_6 + f_2  f_3  f_4 f_5  f_6}$$


## Magnification of Multi Lens System (n = 6)
$$
\begin{align}
M &= m_1 \cdot m_2 \cdot m_3 \cdot m_4 \cdot m_5 \cdot m_6 \\
  &= \left( -\frac{s'_{1}}{s_1} \right)  \left( -\frac{s'_{2}}{s_2} \right) \left( -\frac{s'_{3}}{s_3} \right) \left( -\frac{s'_{4}}{s_4} \right)  \left( -\frac{s'_{5}}{s_5} \right)  \left( -\frac{s'_{6}}{s_6} \right)
\end{align}
$$

Since the initial object has height $h = 2$ m and the image should sbe in the range $h' \in [0.5cm, 3.5cm]$, we conclude that

$$ h' = M \cdot h \Rightarrow M \in [ 0.0025, 0.0175] \Rightarrow M \in [0.25 \%, 1.75\%]$$



In [30]:
F = np.linspace(0.001, 0.5, 1000)
D = np.linspace(0.001, 0.05, 1000)
s1 = 8; s6_ = 0.015; 
eps = 1e-5
n = 6

for f in F :
  for d in D:
    M = 1
    s = s1 
    for _ in range(n - 1):
      s_ = (s*f)/(s - f)
      M *= (- s_/s)
      s = d - s_
    s_ = (s*f)/(s - f)
    M *= (- s_/s)
    if s_ > 0 and abs(s_ - s6_) < eps and 0.0025 <= M <= 0.0175 :
      print("f = {}, d = {} | M = {} s6'  = {}".format(f, d, M, s_))
      print()


f = 0.018482482482482486, d = 0.04955855855855856 | M = 0.002500647269164471 s6'  = 0.014992494277304305

f = 0.027972972972972975, d = 0.013752752752752753 | M = 0.00251272122616987 s6'  = 0.014998450982302692

f = 0.028472472472472476, d = 0.014096096096096099 | M = 0.0025480223399558383 s6'  = 0.014992195520651045

f = 0.029970970970970973, d = 0.015126126126126127 | M = 0.002658079589558576 s6'  = 0.014991941720963981

f = 0.030470470470470475, d = 0.01546946946946947 | M = 0.0026959786938543125 s6'  = 0.014996918441472525

f = 0.030969969969969972, d = 0.015812812812812813 | M = 0.0027344123547398407 s6'  = 0.015003971103108258

f = 0.035465465465465466, d = 0.018951951951951953 | M = 0.0030983451908968645 s6'  = 0.015006694887980053

f = 0.03846246246246247, d = 0.021061061061061065 | M = 0.0033566525237045375 s6'  = 0.015006911322839924

f = 0.04095995995995996, d = 0.02282682682682683 | M = 0.003579220890589726 s6'  = 0.014997440093085606

f = 0.04345745745745746, d = 0.0245925

In [31]:
def plot_lens(x, xi):
  colors = ['r', 'black', 'k', 'blue', 'g', 'y']
  lens = ["1", "2", "3", "4", "5", "6"]
  fig, ax = plt.subplots(figsize = (14,8))
  for i in range(6):
    ax.axvline(x = x[i], color = colors[i])
    ax.axvline(x = xi[i], color = colors[i])
    ax.legend(lens)


# METHOD 2.2 (Flerackers (1984))

### step 1
* $x_j = $ the distance of the j-th lens from the initial object $x_0 = 0$
* $x_{i_{j}} = $ the distance of the image of the j-th lens from the initial object $x_0 = 0$
* $s_j$ = the distance of the j-th lens from the respective object that concerns this lens.
* $s_j'$ = the distance of the j-th lens from the respective image that concerns this lens.

### step 2
Let's find the parameters $A_i, B_i, C_i$ for for all 6 lenses.

$$A_1 = x_1 ^2 \quad A_2 = x_2 ^2 \quad A_3 = x_3 ^2 \quad A_4 = x_4 ^2 \quad A_5 = x_5 ^5 \quad A_6 = x_6 ^2 $$
$$B_1 = x_1 + f \quad B_2 = x_2 + f \quad B_3 = x_3 + f \quad B_4 = x_4 + f \quad B_5 = x_5 + f \quad B_6 = x_6 + f$$
$$C_1 = x_1 - f \quad C_2 = x_2 - f \quad C_3 = x_3 - f \quad C_4 = x_4 - f \quad C_5 = x_5 - f \quad C_6 = x_6 - f$$

\\

$$A_{12} = \frac{A_{2}C_{1} - A_{1}B_{2}}{C_{2} - B_{1}} \qquad B_{12} = \frac{-B_{1}B_{2} + A_{2}}{C_{2} - B_{1}} \qquad C_{12} = \frac{C_{1}C_{2} - A_{1}}{C_{2} - B_{1}} $$


\\

$$A_{123} = \frac{A_{3}C_{12} - A_{12}B_{3}}{C_{3} - B_{12}} \qquad B_{123} = \frac{-B_{12}B_{2} + A_{3}}{C_{3} - B_{12}} \qquad C_{123} = \frac{C_{12}C_{3} - A_{12}}{C_{3} - B_{12}} $$




\\

$$A_{1234} = \frac{A_{3}C_{123} - A_{123}B_{4}}{C_{4} - B_{123}} \qquad B_{1234} = \frac{-B_{123}B_{4} + A_{4}}{C_{4} - B_{123}} \qquad C_{1234} = \frac{C_{123}C_{4} - A_{123}}{C_{4} - B_{123}} $$



\\

$$A_{12345} = \frac{A_{5}C_{1234} - A_{1234}B_{5}}{C_{5} - B_{1234}} \qquad B_{12345} = \frac{-B_{1234}B_{5} + A_{5}}{C_{5} - B_{1234}} \qquad C_{12345} = \frac{C_{1234}C_{5} - A_{1234}}{C_{5} - B_{1234}} $$

\\

$$A_{123456} = \frac{A_{6}C_{12345} - A_{12345}B_{6}}{C_{6} - B_{12345}} \qquad B_{123456} = \frac{-B_{12345}B_{6} + A_{6}}{C_{6} - B_{12345}} \qquad C_{123456} = \frac{C_{12345}C_{6} - A_{12345}}{C_{6} - B_{12345}} $$


### step 3
Let's calculate the $s_{j},s_{j}'$ based on the positions given $x, x_i$


$$ x_{i_{1}} = \frac{A_{1}}{C_{1}} \quad s_{1} = x_{1} \quad s_{1}' =  x_{i_{1}}  - x_{1}$$

\\
$$ x_{i_{2}} = \frac{A_{12}}{C_{12}} \quad s_{2} = x_{2} - x_{i_{1}} \quad s_{2}' =  x_{i_{2}}  - x_{2}$$

\\
$$ x_{i_{3}} = \frac{A_{123}}{C_{123}} \quad s_{3} = x_{3} - x_{i_{2}} \quad s_{3}' =  x_{i_{3}}  - x_{3}$$


\\
$$ x_{i_{4}} = \frac{A_{1234}}{C_{1234}} \quad s_{4} = x_{4} - x_{i_{3}} \quad s_{4}' =  x_{i_{4}}  - x_{4}$$


\\
$$ x_{i_{5}} = \frac{A_{12345}}{C_{12345}} \quad s_{5} = x_{5} - x_{i_{4}} \quad s_{5}' =  x_{i_{5}}  - x_{5}$$

\\
$$ x_{i_{6}} = \frac{A_{123456}}{C_{123456}} \quad s_{6} = x_{6} - x_{i_{5}} \quad s_{6}' =  x_{i_{6}}  - x_{6}$$


In [32]:
eps = 1e-5
n = 6
s1 = 8; s6_ = 0.015; 
F = np.linspace(0.001, 0.5, 1000)
D = np.linspace(0.001, 0.05, 1000)
Ss2 = []
Ms2 = []

for f in F :
  for d in D:
    x = [s1 + i * d for i in range(6)]
    xi, s, s_ = [0] * 6, [0] * 6, [0] * 6
    A = [i ** 2 for i in x]
    B = [i + f for i in x]
    C = [i - f for i in x] 

    xi[0] = A[0]/C[0]; s[0]  = x[0]; s_[0] = xi[0] - x[0]

    A_ = (A[1]*C[0] - A[0]*B[1]) / (C[1] - B[0])
    B_ = (-B[1]*B[0] + A[1]) / (C[1] - B[0])
    C_ = (C[1]*C[0] - A[0]) / (C[1] - B[0])

    xi[1] = A_/C_; s[1]  = x[1] - xi[0]; s_[1] = xi[1] - x[1]

    M = (-s_[0]/s[0]) * (-s_[1]/s[1])

    for i in range(2, 6) :
      den = C[i] - B_
      A_old = A_
      A_ = (A[i] * C_ - A_old * B[i]) / den
      B_ = (-B[i]*B_ + A[i]) / den
      C_ = (C[i]*C_ - A_old) / den
      xi[i] = A_/C_; s[i]  = x[i] - xi[i-1]; s_[i] = xi[i] - x[i]
      M *= (-s_[i]/s[i])
    Ms2.append(M)

    if abs(s_[5] - s6_) < eps and 0.0025 <= M <= 0.0175 : 
      print("f = {}, d = {} | M = {} s6'  = {}".format(f, d, M, s_[5]))
      print()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


f = 0.018482482482482486, d = 0.04955855855855856 | M = 0.002500647268762391 s6'  = 0.014992494277839441



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars


f = 0.027972972972972975, d = 0.013752752752752753 | M = 0.0025127212259423684 s6'  = 0.014998450982313116

f = 0.028472472472472476, d = 0.014096096096096099 | M = 0.002548022339952563 s6'  = 0.01499219552075104

f = 0.029970970970970973, d = 0.015126126126126127 | M = 0.0026580795895398256 s6'  = 0.014991941720829516

f = 0.030470470470470475, d = 0.01546946946946947 | M = 0.002695978694031383 s6'  = 0.014996918441468665

f = 0.030969969969969972, d = 0.015812812812812813 | M = 0.0027344123545528696 s6'  = 0.015003971103270786

f = 0.035465465465465466, d = 0.018951951951951953 | M = 0.0030983451909001886 s6'  = 0.015006694887995664

f = 0.03846246246246247, d = 0.021061061061061065 | M = 0.003356652523781279 s6'  = 0.015006911322776872

f = 0.04095995995995996, d = 0.02282682682682683 | M = 0.0035792208906829715 s6'  = 0.01499744009304571

f = 0.04345745745745746, d = 0.024592592592592596 | M = 0.0038071407721130537 s6'  = 0.014992174322834728

f = 0.048951951951951955, d = 0.028467